## Thinkful exercise 3.2.5 - Dimensionality reduction using random forest model

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Project description

Data:  2015 data from Lending Club, found at this link:https://www.lendingclub.com/info/download-data.action

Modelling objective: Predict the state of a loan given some information about it

Assignment details: 

1) Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

2) Identify which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

3) Can you [answer the question] without using anything related to payment amount or outstanding principal? How do you know?


### Preprocessing data

In [4]:
y2015 = pd.read_csv(
    '../../Datafiles/unit_3/LoanStats3d.csv',
    skipinitialspace=True,
    header=1
)

/Users/robin/anaconda3/envs/data_sci/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
y2015.shape

(421097, 111)

In [6]:
y2015.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139.0,16000.0,16000.0,16000.0,60 months,14.85%,379.39,C,C5,...,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
1,68354783,73244544.0,9600.0,9600.0,9600.0,36 months,7.49%,298.58,A,A4,...,0.0,2.0,100.0,66.7,0.0,0.0,88635.0,55387.0,12500.0,75635.0
2,68466916,73356753.0,25000.0,25000.0,25000.0,36 months,7.49%,777.55,A,A4,...,0.0,0.0,100.0,20.0,0.0,0.0,373572.0,68056.0,38400.0,82117.0
3,68466961,73356799.0,28000.0,28000.0,28000.0,36 months,6.49%,858.05,A,A2,...,0.0,0.0,91.7,22.2,0.0,0.0,304003.0,74920.0,41500.0,42503.0
4,68495092,73384866.0,8650.0,8650.0,8650.0,36 months,19.89%,320.99,E,E3,...,0.0,12.0,100.0,50.0,1.0,0.0,38998.0,18926.0,2750.0,18248.0


In [7]:
y2015.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
421092,36271333,38982739.0,13000.0,13000.0,13000.0,60 months,15.99%,316.07,D,D2,...,0.0,3.0,100.0,50.0,1.0,0.0,51239.0,34178.0,10600.0,33239.0
421093,36490806,39222577.0,12000.0,12000.0,12000.0,60 months,19.99%,317.86,E,E3,...,1.0,2.0,95.0,66.7,0.0,0.0,96919.0,58418.0,9700.0,69919.0
421094,36271262,38982659.0,20000.0,20000.0,20000.0,36 months,11.99%,664.20,B,B5,...,0.0,1.0,100.0,50.0,0.0,1.0,43740.0,33307.0,41700.0,0.0
421095,Total amount funded in policy code 1: 6417608175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421096,Total amount funded in policy code 2: 1944088810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

In [9]:
#One way to remove columns with more nulls than a specified value
y2015 = y2015.loc[:, y2015.isnull().mean() < .76]

In [10]:
y2015.shape

(421095, 92)

In [11]:
#Compute proportion of null values 
null_counts = pd.DataFrame(y2015.isna().mean().round(4) * 100, columns = ['null_pcnt'])

In [12]:
null_counts.index.name = 'col_name'
null_counts = null_counts.reset_index()

In [13]:
print(null_counts.loc[null_counts.null_pcnt > 0])

                          col_name  null_pcnt
10                       emp_title       5.67
11                      emp_length       5.66
20                           title       0.03
27          mths_since_last_delinq      48.44
31                      revol_util       0.04
43                    last_pymnt_d       0.07
45                    next_pymnt_d      27.82
48     mths_since_last_major_derog      70.85
57                  bc_open_to_buy       0.94
58                         bc_util       1.00
61              mo_sin_old_il_acct       2.91
66            mths_since_recent_bc       0.90
67        mths_since_recent_bc_dlq      74.21
68           mths_since_recent_inq      10.59
69  mths_since_recent_revol_delinq      63.97
80                num_tl_120dpd_2m       4.57
85                percent_bc_gt_75       1.01


In [ ]:
# drop_col_list = []
# drop_col_list = null_counts.loc[(null_counts.null_pcnt >= 25.0)].col_name
# print(drop_col_list)

In [ ]:
#y2015 = y2015.drop(columns = drop_col_list)

In [ ]:
#y2015.shape

In [ ]:
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

In [ ]:
# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'id'], 1, inplace=True)

In [ ]:
# Convert ID and Interest Rate to numeric.
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

In [16]:
y2015.head()

col_name,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139.0,16000.0,16000.0,16000.0,60 months,14.85%,379.39,C,C5,...,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
1,68354783,73244544.0,9600.0,9600.0,9600.0,36 months,7.49%,298.58,A,A4,...,0.0,2.0,100.0,66.7,0.0,0.0,88635.0,55387.0,12500.0,75635.0
2,68466916,73356753.0,25000.0,25000.0,25000.0,36 months,7.49%,777.55,A,A4,...,0.0,0.0,100.0,20.0,0.0,0.0,373572.0,68056.0,38400.0,82117.0
3,68466961,73356799.0,28000.0,28000.0,28000.0,36 months,6.49%,858.05,A,A2,...,0.0,0.0,91.7,22.2,0.0,0.0,304003.0,74920.0,41500.0,42503.0
4,68495092,73384866.0,8650.0,8650.0,8650.0,36 months,19.89%,320.99,E,E3,...,0.0,12.0,100.0,50.0,1.0,0.0,38998.0,18926.0,2750.0,18248.0


### Thinkful baseline model

In [ ]:
#Drop dates


In [ ]:
# from sklearn import ensemble
# from sklearn.model_selection import cross_val_score

# rfc = ensemble.RandomForestClassifier()
# X = y2015.drop('loan_status', 1)
# Y = y2015['loan_status']
# X = pd.get_dummies(X)
# X = X.dropna(axis=1)

# cross_val_score(rfc, X, Y, cv=10)

Thinkful commentary:

The score cross validation reports is the accuracy of the tree. Here we're about 98% accurate.

That works pretty well, but there are a few potential problems. Firstly, we didn't really do much in the way of feature selection or model refinement. As such there are a lot of features in there that we don't really need. Some of them are actually quite impressively useless.

There's also some variance in the scores. The fact that one gave us only 93% accuracy while others gave higher than 98 is concerning. This variance could be corrected by increasing the number of estimators. That will make it take even longer to run, however, and it is already quite slow.

### Robin's modeling

#### Create readable list of remaining columns

In [21]:
cols_df = pd.DataFrame(list(y2015.columns.values))

In [14]:
y2015.loan_status.value_counts()

Current               287414
Fully Paid             87989
Charged Off            29178
Late (31-120 days)      9510
In Grace Period         4320
Late (16-30 days)       1888
Default                  796
Name: loan_status, dtype: int64

In [15]:
y2015.grade.value_counts()

C    120567
B    117606
A     73336
D     62654
E     34948
F      9817
G      2167
Name: grade, dtype: int64

In [17]:
sel_cols = [col for col in y2015.columns if 'mths_' in col]

In [18]:
print(sel_cols)

['mths_since_last_delinq', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 'mths_since_recent_revol_delinq']


In [16]:
print(y2015.mths_since_last_delinq.describe())


count    217133.000000
mean         34.023391
std          21.990270
min           0.000000
25%          15.000000
50%          31.000000
75%          50.000000
max         176.000000
Name: mths_since_last_delinq, dtype: float64


Separate collections_12... into collect_none, collect_1_2, collect_gt2

If months since last delinq = 0, means > = 0, < 30 days. So if zero, set no_delinq = True,  else False.

Separate months since last delinq into recent delinquencies -- delinq_last_12_mo = True if >0, <= 12 else false

Then drop cols in sel_cols list

In [25]:
#Use apply to set no_delinq
#y2015['no_delinq'] = np.where(y2015.mths_since_last_delinq == 0, True, False)
y2015['no_delinq'] = np.where((y2015.mths_since_last_delinq == 0) | (y2015.mths_since_last_delinq.isnull()), True, False)
y2015['recent_delinq'] = np.where(((y2015.mths_since_last_delinq > 0) & (y2015.mths_since_last_delinq <= 12)), True, False)

In [26]:
y2015['no_delinq'].value_counts()

False    216633
True     204462
Name: no_delinq, dtype: int64

In [27]:
y2015['recent_delinq'].value_counts()

False    378100
True      42995
Name: recent_delinq, dtype: int64

In [25]:
#Use apply to set no_delinq
#y2015['no_delinq'] = np.where(y2015.mths_since_last_delinq == 0, True, False)
y2015['no_derog'] = np.where((y2015.mths_since_last_major_derog == 0) | (y2015.mths_since_last_delinq.isnull()), True, False)
y2015['recent_derog'] = np.where(((y2015.mths_since_last_delinq > 0) & (y2015.mths_since_last_delinq <= 12)), True, False)

In [ ]:
assert pd.notnull(y2015).all().all()

In [ ]:
# from pandas import ExcelWriter
# from pandas import ExcelFile

# writer = ExcelWriter('../../Datafiles/unit_3/y2015_rmng_cols.xlsx')
# cols_df.to_excel(writer,'cols_df',index=False)
# writer.save()

#### Step 1. Normalize variance and drop columns with low variance

In [ ]:
y2015.shape

In [ ]:
num_y2015 = pd.DataFrame(y2015.select_dtypes(include=['float64', 'int64']))

In [ ]:
from sklearn.feature_selection import VarianceThreshold 

sel = VarianceThreshold(threshold=0.005)
sel.fit(num_y2015 / num_y2015.mean())

mask = sel.get_support() 
reduced_num_y2015 = num_y2015.loc[:, mask] 
print(reduced_num_y2015.shape)

In [ ]:
cols_red_df = pd.DataFrame(list(reduced_num_y2015.columns.values))
writer_reduced = ExcelWriter('../../Datafiles/unit_3/y2015_reduced_cols.xlsx')
cols_red_df.to_excel(writer,'cols_red_df',index=False)
writer.save()

In [ ]:
#pd.get_dummies(y2015)